# ACA Premium Tax Credit Analysis - Tampa Family of 4
## Comparing 600% FPL Cliff Extension and IRA Extension

In [14]:
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from policyengine_core.charts import format_fig

## Define Reforms

### Reform 1: 600% FPL Cliff Extension
Activates the 600% FPL cliff extension parameter.

### Reform 2: IRA Extension
Modifies required contribution percentages:
- Sets 0% for income brackets 0-2
- Sets 2%, 4%, 6%, and 8.5% for higher brackets
- Extends PTC eligibility beyond 400% FPL

In [15]:
# Reform 1: 600% FPL Cliff Extension
reform_600fpl = Reform.from_dict(
    {
        "gov.contrib.aca.ptc_600_fpl_cliff.in_effect": {
            "2026-01-01.2100-12-31": True
        }
    },
    country_id="us",
)

# Reform 2: IRA Extension (from existing notebook reform3)
reform_ira = Reform.from_dict({
    "gov.aca.required_contribution_percentage[0].amount": {
        "2026-01-01.2100-12-31": 0
    },
    "gov.aca.required_contribution_percentage[1].amount": {
        "2025-01-01.2100-12-31": 0
    },
    "gov.aca.required_contribution_percentage[2].amount": {
        "2026-01-01.2100-12-31": 0
    },
    "gov.aca.required_contribution_percentage[3].amount": {
        "2026-01-01.2100-12-31": 0.02
    },
    "gov.aca.required_contribution_percentage[4].amount": {
        "2026-01-01.2100-12-31": 0.04
    },
    "gov.aca.required_contribution_percentage[5].amount": {
        "2026-01-01.2100-12-31": 0.06
    },
    "gov.aca.required_contribution_percentage[6].amount": {
        "2026-01-01.2100-12-31": 0.085
    },
    "gov.aca.ptc_income_eligibility[2].amount": {
        "2026-01-01.2100-12-31": True
    }
}, country_id="us")

## Define Florida Family Situation

Family of 4 in Tampa (Hillsborough County, FL):
- 2 parents (age 40)
- 2 children (ages 10 and 8)

In [16]:
situation_florida = {
    "people": {
        "parent1": {
            "age": {"2026": 40}
        },
        "parent2": {
            "age": {"2026": 40}
        },
        "child1": {
            "age": {"2026": 10}
        },
        "child2": {
            "age": {"2026": 8}
        }
    },
    "families": {
        "your family": {
            "members": ["parent1", "parent2", "child1", "child2"]
        }
    },
    "spm_units": {
        "your household": {
            "members": ["parent1", "parent2", "child1", "child2"]
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": ["parent1", "parent2", "child1", "child2"]
        }
    },
    "households": {
        "your household": {
            "members": ["parent1", "parent2", "child1", "child2"],
            "state_name": {"2026": "FL"},
            "county_fips": {"2026": "12057"}  # Hillsborough County (Tampa)
        }
    },
    "marital_units": {
        "parents marital unit": {
            "members": ["parent1", "parent2"]
        }
    },
    "axes": [[
        {
            "name": "employment_income",
            "count": 800,
            "min": 0,
            "max": 200000
        }
    ]]
}

## Create Simulations

In [17]:
# Baseline simulation
simulation_baseline = Simulation(situation=situation_florida)

# 600% FPL Cliff reform
simulation_600fpl = Simulation(situation=situation_florida, reform=reform_600fpl)

# IRA Extension reform
simulation_ira = Simulation(situation=situation_florida, reform=reform_ira)

## Calculate Benefits and Net Income

In [18]:
# Get household-level values
household_income = simulation_baseline.calculate("employment_income", map_to="household", period=2026)

# Baseline
baseline_aca_ptc = simulation_baseline.calculate("aca_ptc", map_to="household", period=2026)
baseline_net_income = simulation_baseline.calculate(
    "household_net_income_including_health_benefits", 
    map_to="household", 
    period=2026
)

# 600% FPL Cliff
reform_600fpl_aca_ptc = simulation_600fpl.calculate("aca_ptc", map_to="household", period=2026)
reform_600fpl_net_income = simulation_600fpl.calculate(
    "household_net_income_including_health_benefits", 
    map_to="household", 
    period=2026
)

# IRA Extension
reform_ira_aca_ptc = simulation_ira.calculate("aca_ptc", map_to="household", period=2026)
reform_ira_net_income = simulation_ira.calculate(
    "household_net_income_including_health_benefits", 
    map_to="household", 
    period=2026
)

## Visualization Setup

In [19]:
# Color scheme
GRAY = "#808080"
BLUE_PRIMARY = "#2C6496"
PURPLE = "#9467BD"

## Chart 1: ACA Premium Tax Credit by Income Level

In [20]:
fig_ptc = go.Figure()

# Baseline
fig_ptc.add_trace(go.Scatter(
    x=household_income,
    y=baseline_aca_ptc,
    mode='lines',
    name='Baseline',
    line=dict(color=GRAY, width=2)
))

# 600% FPL Cliff
fig_ptc.add_trace(go.Scatter(
    x=household_income,
    y=reform_600fpl_aca_ptc,
    mode='lines',
    name='600% FPL Cliff Extension',
    line=dict(color=BLUE_PRIMARY, width=2)
))

# IRA Extension
fig_ptc.add_trace(go.Scatter(
    x=household_income,
    y=reform_ira_aca_ptc,
    mode='lines',
    name='IRA Extension',
    line=dict(color=PURPLE, width=2)
))

fig_ptc.update_layout(
    title='Florida Family of 4 - ACA Premium Tax Credit by Income',
    xaxis_title='Household Income',
    yaxis_title='ACA Premium Tax Credit',
    xaxis=dict(tickformat='$,.0f', range=[0, 200000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000,
    legend=dict(orientation='h')
)

fig_ptc = format_fig(fig_ptc)
fig_ptc.show()

## Chart 2: Health-Adjusted Net Income

In [21]:
fig_net_income = go.Figure()

# Baseline
fig_net_income.add_trace(go.Scatter(
    x=household_income,
    y=baseline_net_income,
    mode='lines',
    name='Baseline',
    line=dict(color=GRAY, width=2)
))

# 600% FPL Cliff
fig_net_income.add_trace(go.Scatter(
    x=household_income,
    y=reform_600fpl_net_income,
    mode='lines',
    name='600% FPL Cliff Extension',
    line=dict(color=BLUE_PRIMARY, width=2)
))

# IRA Extension
fig_net_income.add_trace(go.Scatter(
    x=household_income,
    y=reform_ira_net_income,
    mode='lines',
    name='IRA Extension',
    line=dict(color=PURPLE, width=2)
))

fig_net_income.update_layout(
    title='Florida Family of 4 - Health-Adjusted Net Income',
    xaxis_title='Household Income',
    yaxis_title='Health-Adjusted Net Income',
    xaxis=dict(tickformat='$,.0f', range=[0, 200000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000,
    legend=dict(orientation='h')
)

fig_net_income = format_fig(fig_net_income)
fig_net_income.show()

## Chart 3: Impact on Net Income (Reform - Baseline)

In [22]:
# Calculate deltas
delta_600fpl = reform_600fpl_net_income - baseline_net_income
delta_ira = reform_ira_net_income - baseline_net_income

fig_delta = go.Figure()

# 600% FPL Cliff impact
fig_delta.add_trace(go.Scatter(
    x=household_income,
    y=delta_600fpl,
    mode='lines',
    name='600% FPL Cliff Extension',
    line=dict(color=BLUE_PRIMARY, width=2)
))

# IRA Extension impact
fig_delta.add_trace(go.Scatter(
    x=household_income,
    y=delta_ira,
    mode='lines',
    name='IRA Extension',
    line=dict(color=PURPLE, width=2)
))

fig_delta.update_layout(
    title='Florida Family of 4 - Impact of Premium Tax Credit Reforms',
    xaxis_title='Household Income',
    yaxis_title='Change in Net Income',
    xaxis=dict(tickformat='$,.0f', range=[0, 200000]),
    yaxis=dict(
        tickformat='$,.0f',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='black'
    ),
    height=600,
    width=1000,
    legend=dict(orientation='h')
)

fig_delta = format_fig(fig_delta)
fig_delta.show()

## Chart 4: Marginal Tax Rates

In [23]:
# Create situation for MTR calculation with more points
situation_fl_for_mtr = {
    "people": {
        "parent1": {
            "age": {"2026": 40},
            "employment_income": {"2026": 0},
            "self_employment_income": {"2026": 0}
        },
        "parent2": {
            "age": {"2026": 40},
            "employment_income": {"2026": 0},
            "self_employment_income": {"2026": 0}
        },
        "child1": {
            "age": {"2026": 10},
            "employment_income": {"2026": 0},
            "self_employment_income": {"2026": 0}
        },
        "child2": {
            "age": {"2026": 8},
            "employment_income": {"2026": 0},
            "self_employment_income": {"2026": 0}
        }
    },
    "families": {
        "your family": {
            "members": ["parent1", "parent2", "child1", "child2"]
        }
    },
    "spm_units": {
        "your household": {
            "members": ["parent1", "parent2", "child1", "child2"]
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": ["parent1", "parent2", "child1", "child2"]
        }
    },
    "households": {
        "your household": {
            "members": ["parent1", "parent2", "child1", "child2"],
            "state_name": {"2026": "FL"},
            "county_fips": {"2026": "12057"}
        }
    },
    "marital_units": {
        "parents marital unit": {
            "members": ["parent1", "parent2"]
        }
    },
    "axes": [[
        {
            "name": "employment_income",
            "min": 0,
            "max": 200000,
            "count": 400,
            "period": "2026"
        }
    ]]
}

# Calculate net incomes for MTR
sim_baseline_mtr = Simulation(situation=situation_fl_for_mtr)
sim_600fpl_mtr = Simulation(situation=situation_fl_for_mtr, reform=reform_600fpl)
sim_ira_mtr = Simulation(situation=situation_fl_for_mtr, reform=reform_ira)

household_income_mtr = sim_baseline_mtr.calculate("employment_income", map_to="household", period=2026)
baseline_net_income_mtr = sim_baseline_mtr.calculate(
    "household_net_income_including_health_benefits", 
    map_to="household", 
    period=2026
)
reform_600fpl_net_income_mtr = sim_600fpl_mtr.calculate(
    "household_net_income_including_health_benefits", 
    map_to="household", 
    period=2026
)
reform_ira_net_income_mtr = sim_ira_mtr.calculate(
    "household_net_income_including_health_benefits", 
    map_to="household", 
    period=2026
)

# Function to calculate MTR
def calc_mtr(incomes, net_incomes):
    """Calculate MTR between adjacent income points."""
    mtrs = []
    mtr_incomes = []
    for i in range(len(incomes) - 1):
        income_change = incomes[i + 1] - incomes[i]
        net_change = net_incomes[i + 1] - net_incomes[i]
        if income_change > 0 and not np.isnan(net_incomes[i]) and not np.isnan(net_incomes[i + 1]):
            mtr = 1 - (net_change / income_change)
            mtrs.append(mtr)
            mtr_incomes.append((incomes[i] + incomes[i + 1]) / 2)
    return np.array(mtr_incomes), np.array(mtrs)

baseline_mtr_incomes, baseline_mtrs = calc_mtr(household_income_mtr, baseline_net_income_mtr)
reform_600fpl_mtr_incomes, reform_600fpl_mtrs = calc_mtr(household_income_mtr, reform_600fpl_net_income_mtr)
reform_ira_mtr_incomes, reform_ira_mtrs = calc_mtr(household_income_mtr, reform_ira_net_income_mtr)

# Create MTR chart
fig_mtr = go.Figure()

fig_mtr.add_trace(go.Scatter(
    x=baseline_mtr_incomes,
    y=np.clip(baseline_mtrs, -1.0, 1.0),
    mode='lines',
    name='Baseline',
    line=dict(color=GRAY, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>Baseline MTR: %{y:.1%}<extra></extra>'
))

fig_mtr.add_trace(go.Scatter(
    x=reform_600fpl_mtr_incomes,
    y=np.clip(reform_600fpl_mtrs, -1.0, 1.0),
    mode='lines',
    name='600% FPL Cliff Extension',
    line=dict(color=BLUE_PRIMARY, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>600% FPL MTR: %{y:.1%}<extra></extra>'
))

fig_mtr.add_trace(go.Scatter(
    x=reform_ira_mtr_incomes,
    y=np.clip(reform_ira_mtrs, -1.0, 1.0),
    mode='lines',
    name='IRA Extension',
    line=dict(color=PURPLE, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>IRA MTR: %{y:.1%}<extra></extra>'
))

fig_mtr.update_layout(
    title='Marginal Tax Rate (including health benefits) - Florida Family of 4',
    xaxis_title='Employment Income',
    yaxis_title='Marginal Tax Rate',
    xaxis=dict(
        tickformat='$,.0f',
        range=[0, 200000],
        gridcolor='lightgray',
        showgrid=True
    ),
    yaxis=dict(
        tickformat='.0%',
        range=[-1.0, 1.0],
        gridcolor='lightgray',
        showgrid=True,
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='gray'
    ),
    height=600,
    width=1000,
    hovermode='x unified',
    plot_bgcolor='white',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig_mtr = format_fig(fig_mtr)
fig_mtr.show()

## Key Income Benchmarks Analysis

Calculate PTC at specific FPL percentages (138%, 300%, 400%, 600%)

In [24]:
import copy
import pandas as pd

PERIOD = 2026

def get_tax_unit_fpg(base_situation: dict, period=PERIOD) -> float:
    """Return the tax unit FPG for the given situation/year."""
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)
    
    for person in sit["people"].values():
        person.setdefault("employment_income", {})
        person["employment_income"][str(period)] = 0
    
    sim = Simulation(situation=sit)
    fpg = sim.calculate("tax_unit_fpg", map_to="tax_unit", period=period)[0]
    return float(fpg)

def calc_ptc_for_income(base_situation: dict, income: float, *, reform_to_use=None, period=PERIOD):
    """Return ACA PTC for the given income and year."""
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)
    
    sit["people"]["parent1"]["employment_income"] = {str(period): income}
    
    sim = Simulation(situation=sit, reform=reform_to_use)
    return sim.calculate("aca_ptc", map_to="household", period=period)[0]

# Get FPG for this family
fpg_2026 = get_tax_unit_fpg(situation_florida, period=PERIOD)

# Define income levels to test
percent_targets = {
    "138% FPL": 1.38,
    "300% FPL": 3.00,
    "400% FPL": 4.00,
    "600% FPL": 6.00,
}

rows = []
for label, mult in percent_targets.items():
    inc = round(fpg_2026 * mult, 2)
    rows.append({
        "income_label": f"{label} (${inc:,.2f})",
        "income_usd": inc,
        "ptc_baseline": calc_ptc_for_income(situation_florida, inc, reform_to_use=None, period=PERIOD),
        "ptc_600fpl_cliff": calc_ptc_for_income(situation_florida, inc, reform_to_use=reform_600fpl, period=PERIOD),
        "ptc_ira_extension": calc_ptc_for_income(situation_florida, inc, reform_to_use=reform_ira, period=PERIOD),
    })

ptc_df = pd.DataFrame(rows)
ptc_df

,income_label,income_usd,ptc_baseline,ptc_600fpl_cliff,ptc_ira_extension
0,"138% FPL ($44,367.00)",44367.0,10217.519531,11747.659180,11747.659180
1,"300% FPL ($96,450.00)",96450.0,9173.240234,12992.660156,12992.660156
2,"400% FPL ($128,600.00)",128600.0,0.000000,7848.660156,7848.660156
3,"600% FPL ($192,900.00)",192900.0,0.000000,0.000000,2383.160156
